In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time
import serial

In [3]:
key = "nyVvF841UBPdCn%2BvugXzKE1EEwkhTBIhjhaXOQgC%2FAlD2F0YongS3tfgXgIxbW5lmlkuJ6Szjamn18IPVNMpXA%3D%3D"

In [4]:
# 아두이노 시리얼 포트와 속도 설정
serial_port = "/dev/tty.usbserial-0001"
baud_rate = 115200

# 시리얼 포트 연결
ard = serial.Serial(serial_port, baud_rate)

In [5]:
def getArrivalInfo(arsId):
    # 정류소고유번호(arsId)를 이용하여 해당 정류소의 정보를 가져옴
    html = requests.get('http://ws.bus.go.kr/api/rest/stationinfo/getStationByUid?serviceKey='+key+'&arsId='+arsId).text
    # 정류소명과 버스도착정보목록을 출력
    station_name = root.find('stNm').text
    message = f'======= {station_name} =======\n'
    # 정류소에 도착하는 버스 정보를 가져옴
    items = root.find_all('itemList')
    for item in items:
        # 버스번호, 남은시간 등의 정보를 출력
        bus_route_id = item.find('busRouteId').text
        bus_route_name = item.find('rtNm').text
        predict_msg = item.find('arrmsg1').text  # 예상 도착 메시지
        if predict_msg == '운행종료':
            message += f'{bus_route_name}: Operation ends\n'
        elif predict_msg == '곧 도착':
            message += f'{bus_route_name}: Arriving soon\n'
        else:
            # 예상 도착 시간과 "후"를 분리하여 처리
            time, suffix = predict_msg.split('후')
            time = time.strip()  # 앞뒤 공백 제거
            # "초" 문자열이 포함된 경우 "초"를 "sec"로 변경하여 출력
            if '초' in time:
                time = time.replace('초', 'sec')
            # 예상 도착 시간을 '분'과 '초'로 분리하고 공백을 추가하여 출력
            minutes, seconds = time.split('분')
            minutes = minutes.strip()  # 앞뒤 공백 제거
            message += f'{bus_route_name} : {minutes} MIN {seconds} later\n'
    return message


In [6]:
def main():
    arsIds = ['05251','05140']  # 4개의 정류소고유번호
    directions = ['건대방향', '군자방향']  # 출력할 방향들
    while True:
        for arsId in arsIds:
            message = getArrivalInfo(arsId)
            # 아두이노 웹 서버에 POST 요청 보내기
            # url = 'http://192.168.0.100/'  # 아두이노 웹 서버의 IP 주소
            url = 'http://172.20.10.2/'  # 아두이노 웹 서버의 IP 주소
            # 172.20.10.2
            headers = {'Content-Type': 'text/plain'}
            response = requests.post(url, headers=headers, data=message.encode('utf-8'))
            print('Response:', response.text)
        time.sleep(60)  # 1분 간격으로 API 값을 불러옴

In [18]:
if __name__ == "__main__":
    main()

KeyboardInterrupt: 